In [1]:
import tifffile # reading/writing tiff
import numpy as np
import napari # python based nD image viewer, not necessary
import nd2 # reading nd2 file
import dask # allows reading data in blocks, used for reading large nd2 files

In [2]:
from dask.distributed import Client
client = Client() # using local computer
client # view tasks being processed by dask

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 48,Total memory: 127.25 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:61381,Workers: 8
Dashboard: http://127.0.0.1:8787/status,Total threads: 48
Started: Just now,Total memory: 127.25 GiB
Comm: tcp://127.0.0.1:61436,Total threads: 6
Dashboard: http://127.0.0.1:61437/status,Memory: 15.91 GiB
Nanny: tcp://127.0.0.1:61384,


In [3]:
### cross register 2 data, two different input formats using a common DAPI channel
# fix, ie reference map, in tiff format
# mov, ie the image that will map to the fix, in nd2 format
# just demonstrating two ways to read the data, just need it in npy matrix


# ZCYX.tiff - 4 colors, last one is DAPI
c = 3
fix_path = '//nrs.hhmi.org/liu/SpatialGenomicData/MouseETS5_E65/Round17-24/Time00000_Channel640,561,488,405_Seq0000.ome.tiff'
fix = tifffile.imread(fix_path)[:, 3, :, :].squeeze()

# TZCYX.nd2 - multi-time point, we will only use the first time point. the last channel is DAPI
t = 0
c = 3
mov_path = '//nrs/liu/SpatialGenomicData/MouseETS5_E65/Round1-8/Channel640,561,488,405_Seq0000.nd2'
mov = nd2.imread(mov_path,dask=True)[t,:,c,:,:].compute() # we will only use dask to read big nd2, and then compute values to store in memory

print(fix.shape,mov.shape) # both DAPI data are in npy format, which can be used for in-memory bigstream functions

(341, 2304, 2304) (341, 2304, 2304)


In [ ]:
import napari
# use napari to look at the two DAPI data
# use subsampling of 8,4,4, the images have lots of data, but we don't actually need to see all of the data points

fix_img = fix[::8,::4,::4]
mov_img = mov[::8,::4,::4]

viewer = napari.Viewer()
viewer.add_image(fix_img,colormap='reds',opacity=0.4, blending='additive')
viewer.add_image(mov_img,colormap='blues',opacity=0.4, blending='additive')

# swtich between the channels, or use different colors and decrease opacity to see that they are not exactly aligned

In [5]:
from bigstream.level_set import foreground_segmentation
from scipy.ndimage import zoom, binary_closing, binary_dilation

spacing = np.array([1., .27, .27]) # ZXY; step (ie. spacing between two adjacent z-slices) = 1um; an edge of a square pixel (ie. spacing between the centers of two adjacent pixels) = 0.27um

# toss out some data, this is a low frequency easy thing to solve for
fix_skip = fix[::2, ::4, ::4] # subsampling of the data
fix_skip_spacing = spacing * [2, 4, 4] # put some empty spaces to keep the dimensions same as the original

# find the mask
fix_mask = foreground_segmentation(
    fix_skip, fix_skip_spacing,
    iterations=(40,8,2), # 40 iteration, fewer iteration = bulkier mask, more iteration = 'tighter' mask that matches real data
    shrink_factors=(4,2,1), 
    smooth_sigmas=(8.,4.,2.),
    lambda2=20.,
    mask_smoothing=1, # how smooth/flexible the edges of mask is
    background=0,
)

# basic topological corrections and restore to full size
fix_mask = binary_closing(fix_mask, np.ones((5,5,5))).astype(np.uint8)
fix_mask = binary_dilation(fix_mask, np.ones((5,5,5))).astype(np.uint8)
fix_mask = zoom(fix_mask, np.array(fix.shape) / fix_skip.shape, order=0)


# save mask
tifffile.imwrite('./bigstream_notebook/fix_mask.tiff', fix_mask, imagej=True, metadata={'axes':'ZYX'})

# load mask, if want to re-use later, without having to re-make it
fix_mask = tifffile.imread('./bigstream_notebook/fix_mask.tiff')

In [6]:
######### repeat the same as above for moving data, just substitute inputs ########
mov_skip = mov[::2, ::4, ::4] # change input
mov_skip_spacing = spacing * [2, 4, 4]
mov_mask = foreground_segmentation(
    mov_skip, mov_skip_spacing, # change input
    iterations=(40,8,2),
    shrink_factors=(4,2,1), 
    smooth_sigmas=(8.,4.,2.),
    lambda2=20.,
    mask_smoothing=1,
    background=0,
)
mov_mask = binary_closing(mov_mask, np.ones((5,5,5))).astype(np.uint8) # change input
mov_mask = binary_dilation(mov_mask, np.ones((5,5,5))).astype(np.uint8) # change input
mov_mask = zoom(mov_mask, np.array(mov.shape) / mov_skip.shape, order=0) # change input


In [7]:
### we will again use napari to see if the mask is good, one can do the same by saving mask as tiff
# we will look at the mask of fix image

viewer = napari.Viewer()
viewer.add_image(fix_img,colormap='reds',opacity=0.4, blending='additive')
viewer.add_image(fix_mask[::8,::4,::4],contrast_limits=[0,1], opacity = 0.3)

# see that the mask covers the sample, if not change param in above foreground_segmentation

<Image layer 'Image' at 0x2111321fdf0>

In [8]:
from bigstream.align import alignment_pipeline

# define what type of alignment we want to do
affine_kwargs = {
    'alignment_spacing':2.0,
    'shrink_factors':(2,),
    'smooth_sigmas':(2.,),
    'optimizer_args':{
        'learningRate':0.25,
        'minStep':0.,
        'numberOfIterations':400, 
    },
}
# we will use affine, a non-rigid transformation = translation,rotation,and scaling for xyz
steps = [('affine', affine_kwargs,),] # an option here is to deform: ('deform', deform_kwargs,) see deformable_align function in align.py and configure_irm.py for parameters

# run the actual alignment
affine = alignment_pipeline(
    fix, mov, # fix data first
    spacing, spacing, # fix spacing first, but the spacing is the same in this case
    steps=steps,
    fix_mask=fix_mask,
    mov_mask=mov_mask
)

# save result
np.savetxt('./bigstream_notebook/affine.mat', affine)

# load result
affine = np.loadtxt('./bigstream_notebook/affine.mat')

## this is the transformation matrix for mov data such that after transfomration it aligns with fix data
## this matrix will be used to transform other channels

LEVEL:  0  ITERATION:  0  METRIC:  -0.20807068970575476
LEVEL:  0  ITERATION:  1  METRIC:  -0.21736476478893985
LEVEL:  0  ITERATION:  2  METRIC:  -0.21985458143256356
LEVEL:  0  ITERATION:  3  METRIC:  -0.22368030827383373
LEVEL:  0  ITERATION:  4  METRIC:  -0.22603611141887228
LEVEL:  0  ITERATION:  5  METRIC:  -0.2285113978039655
LEVEL:  0  ITERATION:  6  METRIC:  -0.23131344203289797
LEVEL:  0  ITERATION:  7  METRIC:  -0.2348784039110255
LEVEL:  0  ITERATION:  8  METRIC:  -0.23864996673715916
LEVEL:  0  ITERATION:  9  METRIC:  -0.24125383285427798
LEVEL:  0  ITERATION:  10  METRIC:  -0.24426372296302928
LEVEL:  0  ITERATION:  11  METRIC:  -0.2482625238894479
LEVEL:  0  ITERATION:  12  METRIC:  -0.2517107952216527
LEVEL:  0  ITERATION:  13  METRIC:  -0.25463513929295345
LEVEL:  0  ITERATION:  14  METRIC:  -0.2576546540645059
LEVEL:  0  ITERATION:  15  METRIC:  -0.26104730848242996
LEVEL:  0  ITERATION:  16  METRIC:  -0.2647641032268889
LEVEL:  0  ITERATION:  17  METRIC:  -0.26815311

In [9]:
from bigstream.transform import apply_transform

# apply the transform to the moving data to resample it
aligned = apply_transform(
    fix, mov, spacing, spacing,
    transform_list=[affine,],
)

# aligned tiff can be saved for fiji visualization
# tifffile.imwrite('./bigstream_notebook/aligned.tiff', aligned, imagej=True, metadata={'axes':'ZYX'})


In [10]:
### we can also use napari to compare 3 data: fix, transformed mov, untransformed mov
aligned_img = aligned[::8,::4,::4]

viewer = napari.Viewer()
viewer.add_image(fix_img,colormap='reds', opacity=0.4, blending='additive')
viewer.add_image(mov_img,colormap='blues', opacity=0.4, blending='additive')
viewer.add_image(aligned_img,colormap='greens', opacity=0.4, blending='additive')

# aligned data should look more like the fix data

<Image layer 'aligned_img' at 0x211134c8f70>

In [3]:
## use the same matrix to align the other channels of moving data

from bigstream.transform import apply_transform

## if DAPI looks aligned that means transform matrix is good. We will use the same matrix to transform other channels of mov data
# ZCYX.tiff - 4 colors, last one is DAPI
c = 3
fix_path = '//nrs.hhmi.org/liu/SpatialGenomicData/MouseETS5_E65/Round17-24/Time00000_Channel640,561,488,405_Seq0000.ome.tiff'
fix = tifffile.imread(fix_path)[:, 3, :, :].squeeze()

# TZCYX.nd2 - multi-time point, we will only use the first time point.
mov_path = '//nrs/liu/SpatialGenomicData/MouseETS5_E65/Round1-8/Channel640,561,488,405_Seq0000.nd2' # same input nd2
affine = np.loadtxt('./bigstream_notebook/affine.mat') # same transformation matrix

spacing = np.array([1., .27, .27])

t = 0 # only using the first time point
C = [0,1,2] # we will loop through other 3 channels for t=0
for c in C:
    print(c)
    mov = nd2.imread(mov_path,dask=True)[t,:,c,:,:].compute()
    aligned = apply_transform(
        fix, mov, spacing, spacing,
        transform_list=[affine,], # 'affine' input here is the transformation matrix we got based on DAPI data
    )
    out_path = './bigstream_notebook/aligned_c'+str(c)+'.tiff'
    print(out_path)
    tifffile.imwrite(out_path, aligned, imagej=True, metadata={'axes':'ZYX'})
    
    
    

0
./bigstream_notebook/aligned_c0.tiff
1
./bigstream_notebook/aligned_c1.tiff
2
./bigstream_notebook/aligned_c2.tiff
